In [ ]:
import numpy as np
from scipy.stats import norm, zscore
import pandas as pd
import matplotlib.pyplot as plt
from QuantConnect import *
from QuantConnect.Data.UniverseSelection import *
from QuantConnect.Research import *

qb = QuantBook()

assets = [ "AAPL", "MSFT", "AMZN", "NVDA", "META", "GOOGL", "GOOG", "TSLA",
            "AVGO", "PEP", "COST", "ADBE", "CSCO", "CMCSA", "TMUS", "INTC",
            "AMGN", "AMD", "NFLX", "QCOM", "TXN", "HON", "INTU", "SBUX",
            "ISRG", "MDLZ", "GILD", "ADI", "ADP", "VRTX", "PYPL", "MU",
            "REGN", "BKNG", "LRCX", "PANW", "KLAC", "SNPS", "CDNS", "MELI",
            "ASML", "MAR", "CTAS", "CSX", "ABNB", "ORLY", "FTNT", "NXPI",
            "PCAR", "WDAY", "CPRT", "ADSK", "ROP", "PAYX", "MRVL", "AEP",
            "ROST", "FAST", "ODFL", "KDP", "BIIB", "VRSK", "CTSH", "IDXX",
            "DXCM", "AZN", "KHC", "CSGP", "EXC", "XEL", "WBD", "EA",
            "ILMN", "LULU", "CCEP", "MCHP", "ON", "DDOG", "GFS",
            "ANSS", "BKR", "MNST", "TEAM", "TTD", "CDW", "FANG", "ZS",
            "CRWD", "DLTR", "DASH", "WBA", "SGEN", "TTWO", "CHTR", "PDD",
            "ENPH", "MDB", "SIRI", "LCID", "RIVN", "DOCU", "OKTA", "ZM"]

# assets = [
#     'REGN', 'BKNG', 'ASML', 'NFLX', 'MELI', 'ROP', 'INTU', 'MDB', 'IDXX', 
#     'ADBE', 'TEAM', 'SNPS', 'CHTR', 'KLAC', 'COST', 'META', 'LULU', 'ILMN', 'ISRG', 
#     'ANSS', 'VRTX', 'TSLA', 'WDAY', 'ENPH', 'MSFT', 'ZS', 'CRWD'
# ]

# assets = ['ODFL', 'VRTX', 'INTU', 'NFLX', 'COST', 'WDAY', 'ABNB', 'AMGN', 'TMUS', 'ADI', 'ASML', 'TXN', 'CSGP', 'PAYX', 'KLAC', 'AMZN', 'ANSS', 
# 'MELI', 'DDOG', 'SGEN', 'SNPS', 'NXPI', 'TTWO', 'ADSK', 'ROP', 'GOOG', 'GOOGL', 'ADBE', 'CDNS', 'ENPH', 'IDXX', 'MSFT', 'META', 'BKNG', 'LULU', 
# 'MDB', 'MAR', 'TEAM', 'CTAS', 'ADP', 'CRWD', 'PDD', 'REGN', 'ZS', 'ON', 'AMD', 'FANG', 'AAPL', 'PANW', 'CDW', 'TTD', 'VRSK', 'CHTR', 'FTNT', 'ROST', 
# 'MCHP', 'ISRG', 'QCOM', 'DOCU', 'PYPL', 'DASH', 'DXCM', 'OKTA', 'ILMN', 'SBUX', 'ZM', 'EA', 'DLTR', 'BIIB', 'HON', 'TSLA', 'AVGO']

for i in range(len(assets)):
    qb.add_equity(assets[i],Resolution.DAILY)

history = qb.history(qb.securities.keys(), datetime(2020, 1, 1), datetime(2025, 1, 1), Resolution.DAILY)

# df = pd.DataFrame(history)
# display(df)

ModuleNotFoundError: No module named 'yfinance'

Getting Historical Data:

How to retrieve and display historical price data for a set of financial securities (stocks) using the QuantBook library in a Jupyter Notebook environment. The script fetches data for a specified list of assets over a defined time period and presents it in a tabular format for analysis.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming history and assets are defined
df = history['close'].unstack(level=0)
performance = {}

for asset in sorted(assets):
    # Create a copy of the column to avoid SettingWithCopyWarning
    df_aapl = df[[asset]].copy()
    
    # Calculate moving averages and standard deviations
    moving_averages = df[asset].rolling(30).mean()
    standard_deviations = df[asset].rolling(30).std()
    
    # Assign values using .loc to avoid warnings
    df_aapl.loc[:, "MA30"] = moving_averages
    df_aapl.loc[:, "SD30"] = standard_deviations
    df_aapl.loc[:, "LB"] = df_aapl["MA30"] - df_aapl["SD30"]
    df_aapl.loc[:, "Lower?"] = df_aapl[asset] < df_aapl["LB"]
    
    mean = df_aapl[asset] > df_aapl["MA30"]
    df_aapl.loc[:, "Crossed mean?"] = mean

    trade_open_price = None
    results = []
    trade_open_indices = []  # Store indices where trades are opened
    trade_close_indices = []  # Store indices where trades are closed
    trade = False
    percentage_return = []
    for index, row in df_aapl.iterrows():
        if trade == False and row["Lower?"]:
            trade_open_price = row[asset]
            trade = True
            trade_open_indices.append(index)  # Record the opening index
        if row["Crossed mean?"] and trade:
            results.append(row[asset] - trade_open_price - 2)  # 2 for fees 
            trade = False
            trade_close_indices.append(index)  # Record the closing index
            percent = ((row[asset] - trade_open_price - 2) / trade_open_price) * 100
            percentage_return.append(percent)
    stats = [result for result in results if result > 0]
    if len(results) == 0:
        proportion = 0
    else:
        proportion = len(stats) / len(results)
    
    performance[asset] = (proportion, len(results), percentage_return)

    if asset in ["CRWD", "MD", "ZS"]:
        

        # Plot green triangles for trade openings
        df[asset].plot(kind='line', figsize=(12, 6))
        for idx in trade_open_indices:
            plt.plot(idx, df_aapl.loc[idx, asset], marker='^', color='green', markersize=10)

        # Plot red triangles for trade closings
        for idx in trade_close_indices:
            plt.plot(idx, df_aapl.loc[idx, asset], marker='v', color='red', markersize=10)

        # Customize the plot
        plt.title(f'{asset} Stock Price with Mean Reversion Strategy', fontsize=14)
        plt.xlabel('Time', fontsize=12)
        plt.ylabel('Price ($)', fontsize=12)
        plt.grid(True)
        plt.xticks(rotation=45)

        # Adjust layout
        plt.tight_layout()

        df_aapl.loc[trade_open_indices, "Enter?"] = True
        df_aapl.loc[trade_close_indices, "Exit?"] = True
        df_aapl = df_aapl.fillna(False).drop(columns=['MA30', 'SD30', 'LB', 'Lower?', 'Crossed mean?'])

        portfolio_value = [df_aapl.iloc[0].iloc[0]]
        # portfolio_value = [5000]
        num_shares = 0
        for index, row in df_aapl.iterrows():
            if num_shares > 0:
                portfolio_value.append(num_shares * row[asset])
                if row['Exit?']:
                    num_shares = 0
            else:
                # Not in trade
                portfolio_value.append(portfolio_value[-1])
                if row['Enter?']:
                    num_shares = portfolio_value[-1] / row[asset]
        df_aapl['Portfolio'] = portfolio_value[1:]
        plt.plot(df_aapl['Portfolio'])

        # Display the plot
        plt.show()

        # break
        

# performance

In [ ]:
proportion = {k: v[0] for k, v in performance.items()}
sorted_proportion = dict(sorted(proportion.items(), key=lambda x: x[1], reverse=True))

series = pd.Series(sorted_proportion)
series.plot(kind='bar', figsize=(12, 6))
plt.axhline(y=0.5, color='r', linestyle='--', label='0.5 Threshold')
plt.xticks(rotation=90, fontsize=8)
plt.tight_layout()
plt.ylabel("Proportion of Success")
plt.xlabel("Name of Stocks")
plt.title("Percentage success of Mean Reversion")

plt.show()

assets_above_05 = [key for key, value in sorted_proportion.items() if value > 0.5]
# print(assets_above_05)


In [ ]:
num_trades = {k: v[1] for k, v in performance.items()}
sorted_num_trades = dict(sorted(num_trades.items(), key=lambda x: x[1], reverse=True))

series = pd.Series(sorted_num_trades)
series.plot(kind='bar', figsize=(12, 6))
plt.xticks(rotation=90, fontsize=8)
plt.tight_layout()
plt.ylabel("Number of trades")
plt.xlabel("Name of Stocks")
plt.title("Number of Trades per stock")

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming performance is defined
percentage_return = {k: v[2] for k, v in performance.items()}

# Calculate medians and averages
medians = {k: np.median(v) for k, v in percentage_return.items()}
averages = {k: np.mean(v) for k, v in percentage_return.items()}

# Sort by medians and averages
sorted_medians = dict(sorted(medians.items(), key=lambda x: x[1], reverse=True))
sorted_averages = dict(sorted(averages.items(), key=lambda x: x[1], reverse=True))

# Extract data for plotting
stocks_median = list(sorted_medians.keys())
values_median = list(sorted_medians.values())
stocks_average = list(sorted_averages.keys())
values_average = list(sorted_averages.values())

# Create subplots (1 row, 2 columns)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 8), sharey=True)

# Plot for median percentage return
ax1.bar(stocks_median, values_median, color='skyblue')
ax1.set_xticks(range(len(stocks_median)))
ax1.set_xticklabels(stocks_median, rotation=90)
ax1.set_title('Median Percentage Return vs Stock', fontsize=14)
ax1.set_xlabel('Stock', fontsize=12)
ax1.set_ylabel('Percentage Return Median', fontsize=12)
ax1.grid(True, axis='y', linestyle='--', alpha=0.7)

# Plot for mean percentage return
ax2.bar(stocks_average, values_average, color='skyblue')
ax2.set_xticks(range(len(stocks_average)))
ax2.set_xticklabels(stocks_average, rotation=90)
ax2.set_title('Mean Percentage Return vs Stock', fontsize=14)
ax2.set_xlabel('Stock', fontsize=12)
ax2.set_ylabel('Percentage Return Mean', fontsize=12)  # Corrected ylabel for clarity
ax2.grid(True, axis='y', linestyle='--', alpha=0.7)

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
top_3 = list(sorted_medians.keys())[:3]
bottom_3 = list(sorted_medians.keys())[-3:]

# Create a figure and a set of subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 14)) # Increased figure size for better clarity
axes = axes.flatten()

# Common settings for labels and titles to increase resolution
title_fontsize = 14
label_fontsize = 12
tick_labelsize = 10
bins_count = 15 # Increased bins for more granularity

# Plot histograms for top 3
for idx, i in enumerate(top_3):
    series = pd.Series(percentage_return[i])
    ax = axes[idx]
    series.plot(kind='hist', bins=bins_count, ax=ax, title=f"Percentage Return of {i}",
                color='skyblue', edgecolor='black')
    ax.set_ylabel("Frequency", fontsize=label_fontsize)
    ax.set_xlabel("Percentage Return", fontsize=label_fontsize)
    ax.set_title(f"Percentage Return of {i}", fontsize=title_fontsize) # Set title font size
    ax.tick_params(axis='x', rotation=45, labelsize=tick_labelsize) # Increased x-tick label size
    ax.tick_params(axis='y', labelsize=tick_labelsize) # Increased y-tick label size

# Plot histograms for bottom 3
for idx, i in enumerate(bottom_3):
    series = pd.Series(percentage_return[i])
    ax = axes[idx + 3]
    series.plot(kind='hist', bins=bins_count, ax=ax, title=f"Percentage Return of {i}",
                color='salmon', edgecolor='black')
    ax.set_ylabel("Frequency", fontsize=label_fontsize)
    ax.set_xlabel("Percentage Return", fontsize=label_fontsize)
    ax.set_title(f"Percentage Return of {i}", fontsize=title_fontsize) # Set title font size
    ax.tick_params(axis='x', rotation=45, labelsize=tick_labelsize) # Increased x-tick label size
    ax.tick_params(axis='y', labelsize=tick_labelsize) # Increased y-tick label size

# Adjust layout to prevent overlapping titles/labels
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
classifier = df.le(df.rolling(30).mean() - df.rolling(30).std())
# Boolean mask or condition that identifies whether the most recent closing price of a 
# stock is less than one standard deviation below its 30-day rolling mean therefore undervalued

z_score = df.apply(zscore)[classifier]
# Measures how many standard deviations a data point is from the mean
# A negative z-score indicates the price is below the mean, with larger absolute values suggesting greater deviation.

magnitude = -z_score * df.rolling(30).std() / df.shift(1)
# Represents the size of the deviation in terms of price movement
# giving an idea of how significant the undervaluation might be in monetary terms.

confidence = (-z_score).apply(norm.cdf)
# The estimated probability that the observed deviation (z-score) is correct, derived from a normal distribution
# A higher confidence value (closer to 1) indicates a stronger likelihood that the stock is undervalued, based 
# on the z-score's position in the distribution.

magnitude.fillna(0, inplace=True)
confidence.fillna(0, inplace=True)

weight = confidence - 1 / (magnitude + 1)
weight = weight[weight > 0].fillna(0)
# Weight represents the allocation or proportion of the portfolio assigned to each stock based on calculated confidence
# and magnitude of undervaluation
sum_ = np.sum(weight, axis=1)
for i in range(weight.shape[0]):
    if sum_[i] > 0:
        weight.iloc[i] = weight.iloc[i] / sum_[i]
    else:
        weight.iloc[i] = 0
weight = weight.iloc[:-1]
# All the weights add up to 1 to ensure the portfolio is fully invested
# Weight prioritises decisions with higher confidence and moderate magnitude
# A higher weight = larger allocation of that stock, based on the belief it will yield a positive return

Prepare Data:

This code processes historical stock price data to analyze how much each stock deviates from its norm, enabling the creation of trading insights and weights for a portfolio. The process involves several steps to transform and analyze the data.

Code Explanation:

1. The close price data is extracted from the history DataFrame and unstacked to restructure it into a format where each column represents a ticker symbol.
2. A classifier is created to determine if the most recent price is less than one standard deviation below the 30-day rolling mean
3. The z-score is calculated, standardising the data. Magnitude is derived, measuring the deviation magnitude. Confidence is computed, estimating the probability of the deviation.
4. The magnitude.fillna(0, inplace=True) and confidence.fillna(0, inplace=True) commands replace any missing values with 0 to ensure data completeness.
5. Trading weight determined

Purpose:

This process prepares data for a trading strategy by identifying undervalued stocks (based on deviation from the mean) and assigning portfolio weights to optimize returns, assuming a long-only investment approach.

In [ ]:
ret = pd.Series(index=range(df.shape[0] - 1))

# Iterate over the range of df.shape[0] - 1 to calculate weighted daily returns
for i in range(df.shape[0] - 1):
    # Compute the weighted return for day i by multiplying the weight at index i
    # with the transposed percentage change of the next day (i + 1)    
    ret[i] = weight.iloc[i] @ df.pct_change().iloc[i + 1].T

# Calculate the cumulative product of returns plus 1 to get the total return over time    
total_ret = (ret + 1).cumprod()
# Align the index of the total return series with the weight series index for consistency
total_ret.index = weight.index
# Plot the total return against time
total_ret.plot(title='Strategy Equity Curve', figsize=(15, 10))
final_value = total_ret.iloc[-1]
print(final_value)
plt.show()

Test Hypothesis:

This code evaluates the performance of a trading strategy by utilizing calculated portfolio weights to optimize returns, based on identifying undervalued stocks through deviation from the mean. The process involves computing daily returns, calculating cumulative returns, and visualizing the results to assess the strategy's effectiveness.

Code Explanation:

1. Get the Total Daily Return:
    - ret is created to represent the daily percentage change in closing prices, excluding the last row.
    - The weighted return ret[i] is calculated for each index i, iterating over the range of df.shape[0] - 1.
2. Call prod to Get the Cumulative Return:
    - total_net = total_ret.cumprod() computes the cumulative product to determine the overall growth of the portfolio over time.
3. Set Index for Visualization:
    - total_net.index = weight.index aligns the index of the cumulative return series with the weight series for consistent plotting.
4. Display the Result

Purpose:

This process tests the hypothesis that a trading strategy based on undervalued stock identification and weighted portfolio allocation can yield positive returns. The resulting "Strategy Equity Curve" visualizes the portfolio's performance (cumulative return) over time, providing insight into the strategy's success or areas for optimization.